## 部署和使用ChatGLM2进行中文问答

### SageMaker  Endpoint 部署ChatGLM2
  
[ChatGLM](https://github.com/THUDM/ChatGLM-6B): ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。 ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

[ChatGLM2](https://github.com/THUDM/ChatGLM2-6B)是新版本，提供了更好的性能和效果。

首先，我们使用Sagemaker部署ChatGLM模型。在SageMaker中部署模型的架构如下：
![](imgs/sagemaker_deploy_model.jpg)

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [2]:
# 如果需要，更新sagemaker和 aws python sdk boto3
# !pip install --upgrade boto3
# !pip install --upgrade sagemaker

In [1]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

接下来，我们使用Sagemaker进行模型部署。

In [11]:
from sagemaker.pytorch.model import PyTorchModel

model_name = None
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'60', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

model = PyTorchModel(
    name = model_name,
    model_data = "./model.tar.gz",
    entry_point = 'inference-chatglm2-fastllm.py',
    source_dir = './code-chatglm2',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

dummy


我们使用了bring your own script 的方式部署模型，通过提供一个dummy的模型文件（model.tar.gz），在 *code/inference-chatglm.py* python脚本里面，部署了模型：
```python
def model_fn(model_dir):
    model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).half().cuda()
    return model
```


然后部署该模型为 Sagemaker endpoint

In [12]:

# deploy as sagemaker endpoint
endpoint_name = 'mt-chatglm2-6b-g4dn'
instance_type = 'ml.g4dn.2xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type,
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

-----------!

In [20]:
del_predictor = sagemaker.Predictor(
    endpoint_name="mt-chatglm2-6b-g4dn", 
    sagemaker_session=sess, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer())
del_predictor.delete_model()
del_predictor.delete_endpoint()

In [30]:
inputs= [
    {"ask": "写一首关于交通信号灯的诗"},
    {"ask": "陨石为什么总能落在陨石坑里?" },
    {"ask": "为什么爸妈结婚没叫我参加婚礼?"}
]
response = predictor.predict(inputs[0])
print("\n\n问题: ", inputs[0]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(inputs[1])
print("\n\n问题: ", inputs[1]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(inputs[2])
print("\n\n问题: ", inputs[2]["ask"], "\n回答:\n", response["answer"])



问题:  写一首关于交通信号灯的诗 
回答:
 交通信号灯，闪亮如星辰，   
指引行人走，车辆停。    
红绿黄三种颜色，  
提醒人们要文明。

当红灯亮时，停车等待，   
绿灯亮时，通行顺畅，   
黄灯闪烁，警惕慢行 。  
交通安全是每个人的责任。  

交通信号灯，是我们的保障，   
让道路通行更加有序，   
让我们的出行更加安全。


问题:  陨石为什么总能落在陨石坑里? 
回答:
 并非陨石总能落在陨石坑里,但陨石坑确实是一个常见的陨石撞击地点。陨石坑是由撞击地球表面的小行星、彗星或陨石等天体形成的。当这些天体进入地球大气层时,由于与大气分子的摩擦而产生高温和高压,导致它们开始燃烧和蒸发,形成一个明亮的火球。

当这些火球与地面接触时,由于陨石的质量和速度,它们会产生巨大的冲击力,导致地表上形成一个陨石坑。由于陨石坑本身的特征和地理位置等因素的影响,陨石坑内的陨石往往会被保存下来,并成为研究陨石撞击地球表面历史的重要样本。


问题:  为什么爸妈结婚没叫我参加婚礼? 
回答:
 很多原因可能导致父母没有邀请你参加他们的婚礼。以下是一些可能的原因：

1. 你的太小：如果你的父母可能认为你太小，不方便参加婚礼或者可能影响你的日常生活。

2. 太忙：你的父母可能当时太忙了，无法请假或者安排时间参加婚礼。

3. 距离：如果你的父母住在较远的地方，他们可能认为参加婚礼会给你带来不必要的麻烦。

4. 不确定：你的父母可能认为没有必要邀请你参加他们的婚礼，或者他们可能认为你已经长大，可以自己处理自己的生活。

5. 预算：你的父母可能认为邀请你参加婚礼会增加他们的负担，因为他们可能需要支付更多的费用。

无论原因是什么，如果父母没有邀请你参加他们的婚礼，你可以尝试和他们沟通，了解他们的想法。也许你可以通过沟通，让他们了解你的想法，并找到一种适合大家的解决方案。


### ChatGLM2 测试
下面，我们使用真格资金建立的一套中文语言模型的测试集，来对这个模型进行测试：
https://github.com/zhenbench/z-bench

真格基金提供了3个级别的测试集，基础能力、进阶能力和垂直能力，并且提供了使用 GPT3.5 和 GPT4 的参考答案，可以用这个测试集进行问答测试，并跟GPT的结果进行比较，来检验语言模型的能力。

#### 基础能力
首先来看看基础能力：

In [31]:
import pandas as pd
benchdf_common = pd.read_csv("./z-bench/common.samples.csv")

In [32]:
benchdf_common.shape

(63, 5)

In [33]:
import random
r = random.randint(0, 63)
test_data = benchdf_common.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"], "params": {"temperature": 0.99}})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `扩写` `逻辑关系` 参考回答: nan
Prompt: 根据这些笔记写一篇餐厅评论： 餐厅名称：The Blue Wharf Lobster 很棒，很吵，服务很有礼貌，价格不错。
ChatGLM: The Blue Wharf Lobster is an excellent choice for a casual dining experience. The atmosphere is lively and the service is top-notch. The lobster is well-prepared and the price is more than just reasonable.

The moment you step into the restaurant, you will be greeted by the warm and inviting ambiance. The high ceilings and the bustling atmosphere make it an ideal place to enjoy a casual meal with friends or family. The service is nothing short of excellent, with staff who are always available to take your order and make sure your needs are met.

As for the food, The Blue Wharf Lobster is known for its delicious lobster. The lobster is prepared to perfection and the seasonings used are top-notch. The mussels and clams are also excellent choices and are prepared with the utmost care.

Overall, The Blue Wharf Lobster is a great choice for those looking for a casual dining experience.

#### 进阶能力

In [34]:
import pandas as pd
benchdf_emergent = pd.read_csv("./z-bench/emergent.samples.csv")

In [35]:
benchdf_emergent.head()

,Prompt,任务类型,参考回答,GPT3.5,GPT4
0,如果我在空中垂直抛球，球最初向哪个方向行进？,`物理直觉`,向上,当你在空中垂直抛球时，球最初行进的方向是向上的，也就是和地面垂直的方向。在球到达最高点时，它...,当你在空中垂直向上抛球时，球最初的方向是向上。它将沿着垂直方向受到重力的作用而减速，直至在其...
1,金元素属于哪一种化学键？,`物理直觉`,金属键,金是一种金属元素，因此通常会形成金属键。金属键是由金属元素组成的化学键，这些金属元素的原子之...,金属元素，如金（Au），通常与其他金属元素通过金属键（metallic bond）相互连接。...
2,假设一个原子发射一个光子，原子会发生什么？,`物理直觉`,失去能量,当一个原子发射一个光子时，它会处于一个激发态或高能态，并且会向低能态或基态释放能量。在这个过...,当一个原子发射一个光子时，这意味着原子中的一个电子从一个较高的能级跃迁到一个较低的能级。在这...
3,假设一辆车可以在 3.85s 的时间内从 0 加速到 27.8 m/s，请计算这辆车的加速度...,`物理`,7.22 m/s/s,根据运动学公式，加速度等于速度变化量除以时间。那么，这辆车的加速度为：a = (27.8 m...,为了计算加速度，我们可以使用以下公式：加速度（a）= (最终速度（vf） - 初始速度（vi...
4,Zdeno Chara 以 175.1 km/h 的速度创造了最快的冰球拍击记录。假设一个冰...,`物理`,a) 8.28 kg·m/s b) Chara 的拍击速度为 175.1 km/h = 48...,a) Chara 的拍击球的动量为：p = mv其中，m = 0.17 kg 是冰球的质量，...,a) 要计算冰球的动量，我们使用动量公式：p = m * v，其中 p 为动量，m 为质量，...


In [36]:
import random
r = random.randint(0, 209)
test_data = benchdf_emergent.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"]})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `ToM` `心理学` 参考回答: 萨莉会去篮子里找小球。
Prompt: 萨莉把一个小球放到篮子里，然后用布将篮子盖起来后离开。安娜在萨莉走后，将小球从篮中取出放进身边的盒子里。过一会儿，萨莉回来，萨莉会到哪里去找小球呢?
ChatGLM: 萨莉可能会回到篮子所在的位置,或者根据她的经验或者直觉来寻找小球。她可能会检查篮子周围是否有留下小球的痕迹或者留下任何线索,比如小球被放在哪里、是否被别人移动过等等。如果萨莉没有找到小球,她可能会询问其他人或者再次回到篮子所在的位置寻找。
GPT3.5: 萨莉会去篮子里找小球。因为萨莉不知道安娜已经将小球取出来放到盒子里了，她认为小球仍然在篮子里。
GPT4: 当萨莉回来时，她会在篮子里找小球，因为在她离开之前，她把小球放在了那里。然而，实际上小球已经被安娜移到了盒子里。


### 垂直类的问题

In [37]:
import pandas as pd
benchdf_specialized = pd.read_csv("./z-bench/specialized.samples.csv")
benchdf_specialized.shape

(39, 5)

In [38]:
import random
r = random.randint(0, 39)
test_data = benchdf_specialized.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"]})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `生物` 参考回答: d
Prompt: 同源结构经常被引用作为自然选择过程的证据。以下所有的结构都是同源结构，除了
(A) 鸟的翅膀和蝙蝠的翅膀
(B) 鲸鱼的鳍和人的手臂
(C) 海豚的胸鳍和海豹的鳍
(D) 昆虫的前腿和狗的前肢
ChatGLM: 答案是 (D) 昆虫的前腿和狗的前肢，因为它们不属于同源结构。鸟的翅膀和蝙蝠的翅膀、鲸鱼的鳍和人的手臂、海豚的胸鳍和海豹的鳍都是同源结构，而昆虫的前腿和狗的前肢不是同源结构。
GPT3.5: 选项 D 昆虫的前腿和狗的前肢不是同源结构。
GPT4: (A) 鸟的翅膀和蝙蝠的翅膀
同源结构是指具有相似结构、起源和发育过程的生物器官，但在不同物种中可能具有不同的功能。这些相似结构表明这些物种可能拥有共同的祖先。鸟的翅膀和蝙蝠的翅膀并非同源结构，而是类似功能的异源结构。虽然它们都是飞行器官，但它们的骨骼结构和起源并不相同。鸟类翅膀的骨骼基于前肢，而蝙蝠的翅膀由手指延长和皮膜连接而成。这种相似性是由于趋同进化，即不同物种独立地演化出相似的适应特征。


### 通过Sagemaker Endpoint调用
我们已经将模型部署到了Sagemaker Endpoint上，我们就可以通过这个Endpoint名称，来调用模型进行推理，这样即使你停止了这个notebook，也能使用已经部署的模型。

In [16]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
endpoint_name = 'mt-chatglm2-6b-g4dn'

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text

payload = {"ask": "将下面的段落使用书面语改写，并进行适当的概括：\n", "parameters": {"max_length": 15000}, "history": []}

In [18]:
%%time
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

国家名:阿根廷
人名:梅西
CPU times: user 4.8 ms, sys: 0 ns, total: 4.8 ms
Wall time: 559 ms


In [24]:
%%time
payload = {"ask": "天为什么是蓝色的？"}
answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射,这些颗粒包括氧气、氮气和水蒸气等。这些分子和颗粒吸收较短波长的光,如紫色和蓝色,而较长波长的光,如红色和橙色则被散射得更少。

由于蓝色光的波长比红色光短,因此它更容易被散射,而在大气层中被散射的蓝色光会向各个方向散射,包括朝着我们的眼睛,因此天空呈现出蓝色。在日出和日落时,太阳光穿过更长的大气层路径,较多的光被散射,因此天空呈现出更温暖的色调,如红色、橙色和黄色。
233
CPU times: user 5.07 ms, sys: 0 ns, total: 5.07 ms
Wall time: 8.53 s


In [19]:
%%time
payload = {"ask": "4加2等于几？"}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

4加2等于6。
CPU times: user 5.53 ms, sys: 0 ns, total: 5.53 ms
Wall time: 446 ms


In [9]:
%%time
payload = {
    "ask": "再乘以3呢？",
    "history": [("数学计算：\n3加8等于几？\n答案：", "3加8等于11。")]}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

如果你要将11乘以3,答案是33。
CPU times: user 4.07 ms, sys: 0 ns, total: 4.07 ms
Wall time: 817 ms


### 删除 EndPoint

In [13]:
predictor.delete_model()
predictor.delete_endpoint()